In [ ]:
#All our files are in the google drive folder and we are mounting our google drive here
from google.colab import drive
drive.mount("/content/gdrive")
# %ls
%cd 'gdrive/Shareddrives/NLP'
# %cd ..
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/Shareddrives/NLP'
/content/gdrive/Shareddrives/NLP
'Anmol_dongrun_jia(odd ratio for all 25 stories).ipynb'   odd_ratio_male.csv
'Books _DataSheet - Sheet1.csv'                           odds_female.csv
'Copy of Copy of nlp_project_1.ipynb'                     odds_male.csv
'Copy of nlp_project_1.ipynb'                             Ref_nlp_project.ipynb
 Dongrun.gdoc                                             stories/
'dongrun_jia(odd ratio for all 25 stories).ipynb'         stories_events/
 nlp_project.ipynb                                        Untitled0.ipynb
 odd_ratio_female.csv


In [ ]:
#Importing important packages
import spacy
import nltk
from nltk import tokenize
import pandas as pd

# Pre-processing

In [ ]:
#Loading our stories into a dataframe from our books csv file
stories = pd.read_csv('Books _DataSheet - Sheet1.csv',sep=',',error_bad_lines=False)
stories.head()

,Name of the Book,AUthor,Author Gender,Genre,HTML Link to the book,Unnamed: 5,Done by,https://www.gutenberg.org/browse/scores/top#authors-last30
0,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,Female,Fiction,https://www.gutenberg.org/ebooks/84.html.images,NaN,Poorvi,NaN
1,Pride and Prejudice,Jane Austen,Female,Classics,https://www.gutenberg.org/ebooks/1342.html.images,NaN,Poorvi,NaN
2,The Scarlet Letter,Nathaniel Hawthorne,Male,Fiction,https://www.gutenberg.org/ebooks/25344.html.im...,NaN,Poorvi,NaN
3,A Christmas Carol in Prose; Being a Ghost Stor...,Charles Dickens,Male,Children’s Literature,https://www.gutenberg.org/ebooks/46.html.images,NaN,Poorvi,NaN
4,Alice's Adventures in Wonderland,Lewis Carroll,Male,Children’s Literature,https://www.gutenberg.org/ebooks/11.html.images,NaN,Poorvi,NaN


In [ ]:
#Extracting stories from the .HTML file
nltk.download('punkt')
import requests
import re
from bs4 import BeautifulSoup
sentences=[]
author_gender=[]
genre=[]
#As our texts are stored as <p> we are extracting only <p> tags from the file
for index,story in stories.iterrows():
  page = requests.get(story['HTML Link to the book'])
  soup = BeautifulSoup(page.content, 'html.parser')
  trial = soup.find_all('p')
  paragraphs=[]
  for p in trial:
    paragraphs.append(str(p))
  paragraphs_clean=[]
  #Using the BeautifulSoup library to parse through the string and remove any html tags and other characters except alphabets
  for p in paragraphs:
    removed_htmltag_sent = BeautifulSoup(p, "html.parser").getText()
    paragraphs_clean.append(re.sub('[^a-zA-Z. ]','', str(removed_htmltag_sent)))
  for p in paragraphs_clean:
    temp = nltk.tokenize.sent_tokenize(p)
    for s in temp:
      sentences.append(s)
      author_gender.append(story['Author Gender'])
      genre.append(story['Genre'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(len(sentences))

152437


In [ ]:
#Final dataframe with the sentences of the stories, author gender and story genre
data = pd.DataFrame({'Sentences':sentences,'Author':author_gender,'Genre':genre})
display(data)

,Sentences,Author,Genre
0,To Mrs. Saville England.,Female,Fiction
1,St. Petersburgh Dec. th .,Female,Fiction
2,You will rejoice to hear that no disaster has ...,Female,Fiction
3,I arrived here yesterday and my first task is ...,Female,Fiction
4,I am already far north of London and as I walk...,Female,Fiction
...,...,...,...
152432,Of Silver we have heard no more.,Male,Children’s Literature
152433,That formidable seafaring man with one leg has...,Male,Children’s Literature
152434,It is to be hoped so I suppose for his chances...,Male,Children’s Literature
152435,The bar silver and the arms still lie for all ...,Male,Children’s Literature


# Event Extraction 

In [ ]:
# CLeaned Sentences text from Stories
sentences[:10]

['To Mrs. Saville England.',
 'St. Petersburgh Dec. th .',
 'You will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings.',
 'I arrived here yesterday and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking.',
 'I am already far north of London and as I walk in the streets of Petersburgh I feel a cold northern breeze play upon my cheeks which braces my nerves and fills me with delight.',
 'Do you understand this feeling This breeze which has travelled from the regions towards which I am advancing gives me a foretaste of those icy climes.',
 'Inspirited by this wind of promise my daydreams become more fervent and vivid.',
 'I try in vain to be persuaded that the pole is the seat of frost and desolation it ever presents itself to my imagination as the region of beauty and delight.',
 'There Margaret the sun is for ever visible its broad disk ju

Find Events Using Spacy

In [ ]:
# Finds Events Using Spacy's Rule based Methods (POS tagging + Dependency Parsing)
# For each sentence this section finds the list of pairs of (event, gender attribute)

nlp = spacy.load("en_core_web_sm")

def find_event_with_gender_pronoun1(doc):
    sentence_events = []
    for token in doc:
        if token.pos_ == "VERB":     
          v1 = [str(child).lower() for child in token.children if str(child.pos_) == "PROPN" or str(child.pos_) == "PRON"]
          for v in v1:
            sentence_events.append([token.lemma_, v,])
    return sentence_events
events = []
for i in range(len(sentences)):
    doc = nlp(sentences[i])
    sentence_events = find_event_with_gender_pronoun1(doc)
    if len(sentence_events):
      sentence_events = list(map(lambda x: x + [author_gender[i], genre[i]], sentence_events))
      events += sentence_events

In [ ]:
# Output events to events.txt
for element in events:
  for e in element:
    file1.write(str(e) + " ")
  file1.write("\n")

In [ ]:
print(events[:50])

[['live', 'fogg'], ['die', 'sheridan'], ['seem', 'he'], ['resemble', 'he'], ['resemble', 'byron'], ['live', 'who'], ['see', 'he'], ['enter', 'he'], ['enter', 'lincoln'], ['resound', 'bench'], ['know', 'he'], ['know', 'association'], ['belong', 'he'], ['get', 'he'], ['recommend', 'he'], ['know', 'who'], ['know', 'him'], ['make', 'he'], ['know', 'he'], ['supply', 'he'], ['supply', 'it'], ['talk', 'he'], ['do', 'he'], ['travel', 'he'], ['appear', 'he'], ['correct', 'he'], ['travel', 'he'], ['absent', 'fogg'], ['absent', 'himself'], ['honour', 'who'], ['pretend', 'nobody'], ['see', 'him'], ['win', 'he'], ['play', 'fogg'], ['know', 'fogg'], ['live', 'he'], ['serve', 'him'], ['breakfast', 'he'], ['use', 'he'], ['provide', 'reform'], ['pass', 'he'], ['choose', 'he'], ['breakfast', 'he'], ['serve', 'he'], ['proffer', 'who'], ['confess', 'it'], ['require', 'fogg'], ['require', 'him'], ['dismiss', 'he'], ['dismiss', 'forster']]


[['heard', 'I'], ['mention', 'him'], ['mention', 'her']]

# Gender Categorization


In [ ]:
#Comprehensive lists of all the pronouns we can use to identify male or female characters in the stories
female_list=['lady','female','girl','wife','Señora','gal','sister','sheila','broad','maiden','damsel','gentlewoman','daughter', 'hers', 'grandma', 'grandmother', 'aunt', 'sis', 'niece', 'mother', 'she', 'her', 'granny', 'granddaughter', 'girlfriend', 'woman', 'mom','miss','mrs.','ms']
male_list=['man','gentleman','boy','youth','guy','fellow','gent','bloke','chap','hombre','dude', 'godfather', 'grandson', 'stepbrother', 'sir', 'he', 'uncle', 'male', 'soninlaw', 'boyfriend', 'brother', 'grandpa', 'him', 'nephew', 'son', 'papa', 'exboyfriend', 'granddad', 'husband', 'stepson', 'dad', 'fatherinlaw', 'daddy', 'stepdad', 'father', 'grandfather', 'bro', 'his','Mr.']

In [ ]:
#From the events and pronouns extraccted we are identifying if the gender of the character is a male or a female
events = []
for e in events_list:
  e = re.sub('[^a-zA-Z._]','', str(e))
  events.append(e.split('_'))
print(events[:10])
import pandas as pd
verb=[]
subject=[]
gender=[]
author_gender = []
story_genre = []
for x in events:
  if str(x[1]) in female_list:
    verb.append(x[0])
    subject.append(x[1])
    gender.append('F')
    author_gender.append(x[2])
    story_genre.append(x[3])
  if str(x[1]) in male_list:
    verb.append(x[0])
    subject.append(x[1])
    gender.append('M')
    author_gender.append(x[2])
    story_genre.append(x[3])
df = pd.DataFrame({'Event':verb,'Subject':subject,'Gender':gender, 'Author Gender':author_gender,'Story Genre':story_genre})
print(df.head(20))

[['rejoice', 'you', 'Female', 'Fiction'], ['regard', 'you', 'Female', 'Fiction'], ['arrive', 'i', 'Female', 'Fiction'], ['walk', 'i', 'Female', 'Fiction'], ['feel', 'i', 'Female', 'Fiction'], ['fill', 'me', 'Female', 'Fiction'], ['understand', 'you', 'Female', 'Fiction'], ['advance', 'i', 'Female', 'Fiction'], ['give', 'me', 'Female', 'Fiction'], ['try', 'i', 'Female', 'Fiction']]
       Event Subject Gender Author Gender Story Genre
0     embark      he      M        Female     Fiction
1   consider      he      M        Female     Fiction
2     engage     him      M        Female     Fiction
3     engage     him      M        Female     Fiction
4       love      he      M        Female     Fiction
5        see      he      M        Female     Fiction
6      bathe     she      F        Female     Fiction
7    entreat     him      M        Female     Fiction
8      spare     her      F        Female     Fiction
9       love     she      F        Female     Fiction
10       buy      he  

In [ ]:
#Example - Here we are checking the number of occurences of the event "fight" and the number of occurences of fight associated with a "Female"
df1= df.loc[df['Event'] == 'fight']
print(len(df1))
df2 = df1.loc[df1['Gender'] == 'F']
print(len(df2))

22
2


In [ ]:
#Storing the dataframe in a .csv file
df.to_csv('events_with_details.csv', index=False)